In [1]:
from drawing_bot_api.trajectory_optimizer.simulator import Simulator, ExponentialDecaySimulator
from drawing_bot_api.trajectory_optimizer.shape_generator import ShapeGenerator
from drawing_bot_api.commands import DrawingBot
import numpy as np
import matplotlib.pyplot as plt
from drawing_bot_api.trajectory_optimizer.image_processor import ImageProcessor
from drawing_bot_api.trajectory_optimizer.utils import Scheduler
from IPython.display import clear_output
import signal

DEBUG_MODE = True

%matplotlib agg

np.set_printoptions(threshold=np.inf)

2025-01-27 08:57:16.526247: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
drawing_bot = DrawingBot()
simulator = Simulator()
compensator = ExponentialDecaySimulator()
shape_generator = ShapeGenerator()
image_processor = ImageProcessor()

Unit set to "mm".


In [3]:
def plot_graph(data, labels, scale='linear'):
    plt.yscale(scale)
    for _i in range(len(data)):
        plt.plot(data[_i], label=labels[_i])
    plt.legend(bbox_to_anchor=(1, 1.15), ncol=3)  
    plt.show() 

In [4]:
def get_trajectory():
    for shape in shape_generator():
        drawing_bot.add_shape(shape)
    trajectory = np.array(drawing_bot._get_all_points())
    template = drawing_bot.plot(training_mode=True, points=trajectory)
    drawing_bot.shapes.clear()
    return trajectory, template

In [5]:
def get_parameters(reference_parameters):
    pass

In [6]:
def test_parameters(parameters, trajectory, template):
    compensated_trajectory = compensator(trajectory, gain=parameters[0], damping=parameters[1], freq=parameters[2], non_linearity=parameters[3])
    simulated_trajectory = simulator(compensated_trajectory)
    drawing = drawing_bot.plot(training_mode=True, points=simulated_trajectory)
    reward = image_processor(template, drawing, save_images=False)
    return reward, drawing

In [ ]:
KEEP_SETS = 2
GENERATION_SIZE = 10
GENERATIONS = 300

SIGMA_BASE_INIT = 1.5
SIGMA_DECAY_INIT = 0.98

SIGMA_SWITCH_GENERATIION = 50
SIGMA_BASE_SWITCH = 0.05
SIGMA_DECAY_SWITCH = 0.999

FIXED_GAIN = 1

: 

In [ ]:
best_set = None
parameters = np.random.random((GENERATION_SIZE, 4))
sigma_schedule = Scheduler(SIGMA_BASE_INIT, SIGMA_DECAY_INIT)
reward_history = []

for generation_index in range(GENERATIONS):
    signal.alarm(5)

    trajectory, template = get_trajectory()

    try:
        sigma = sigma_schedule()

        if generation_index % SIGMA_SWITCH_GENERATIION == SIGMA_SWITCH_GENERATIION-1:
            sigma_schedule = Scheduler(SIGMA_BASE_SWITCH, SIGMA_DECAY_SWITCH)

        rewards = []
        drawings = []
        for set in parameters:
            reward, drawing = test_parameters(set, trajectory, template)
            rewards.append(reward)
            drawings.append(drawing)

        reward_history.append(max(rewards))
        
        # Get indices of the three best sets
        rewards = np.nan_to_num(rewards, nan=-np.inf)
        best_indices = np.argsort(rewards)[-KEEP_SETS:]
        best_sets = [parameters[i] for i in best_indices]
        best_set = parameters[np.argmax(rewards)]

        print(f'generation: {generation_index}\tbest_set: {np.argmax(rewards)}\t{max(rewards)}\tsigma: {sigma}\tbest: {best_set}')
        
        parameters = np.random.random((GENERATION_SIZE, 4))
        parameters[0] = best_set
        for i in range(1, GENERATION_SIZE):
            parameters[i] = best_sets[i % KEEP_SETS]
            parameters[i] += np.random.normal(0, sigma, 4)

        if generation_index % 10 == 9:
            image_processor.save_image(drawings[np.argmax(rewards)], 'evolutionary', 'generation', nr=generation_index)

        plt.close('all')
    except Exception as e:
        if DEBUG_MODE:
            raise
        else:
            print(f"Timeout occurred: {e}")

    finally:
        pass
        signal.alarm(0)  # Cancel the alarm

%matplotlib inline

print(f'BEST SET: {best_set}')
plot_graph([reward_history], ['reward'], scale='linear')

compensated_trajectory = compensator(trajectory, gain=best_set[0], damping=best_set[1], freq=best_set[2], non_linearity=best_set[3])
simulated_trajectory = simulator(compensated_trajectory)
drawing = drawing_bot.plot(training_mode=True, points=simulated_trajectory, plot_in_training=True)


generation: 0	best_set: 7	0.6865331990827594	sigma: 1.5	best: [0.07347286 0.72048716 0.23880196 0.65105956]
generation: 1	best_set: 2	0.7112013519883698	sigma: 1.47	best: [ 0.37769761  1.18602176  3.43845563 -1.61938637]
generation: 2	best_set: 2	0.7213184067467722	sigma: 1.4405999999999999	best: [ 0.11003234 -0.81945995  0.1084137   0.33534666]
generation: 3	best_set: 0	0.7026849938009452	sigma: 1.4117879999999998	best: [ 0.11003234 -0.81945995  0.1084137   0.33534666]
generation: 4	best_set: 0	0.68967742614641	sigma: 1.3835522399999998	best: [ 0.11003234 -0.81945995  0.1084137   0.33534666]
generation: 5	best_set: 0	0.7571236081615376	sigma: 1.3558811951999998	best: [ 0.11003234 -0.81945995  0.1084137   0.33534666]
generation: 6	best_set: 7	0.7052209252535009	sigma: 1.3287635712959998	best: [ 0.1852941  -0.34227215 -0.28899843 -1.09450767]
generation: 7	best_set: 2	0.6534491557163566	sigma: 1.30218829987008	best: [ 0.02029872  0.63170278  1.80087474 -2.9225989 ]
generation: 8	best_se

/Users/leon/Documents/Dateien/Privat/Projekte/Bachelorarbeit/Project/MIRMI_2DOF_Drawing_Bot/code/api/drawing_bot_api/trajectory_optimizer/simulator.py:128: RuntimeWarning: divide by zero encountered in divide
  oscillation = (oscillation / np.max(oscillation) + 1e-8) * -gain
/Users/leon/Documents/Dateien/Privat/Projekte/Bachelorarbeit/Project/MIRMI_2DOF_Drawing_Bot/code/api/drawing_bot_api/trajectory_optimizer/simulator.py:128: RuntimeWarning: invalid value encountered in divide
  oscillation = (oscillation / np.max(oscillation) + 1e-8) * -gain
/Users/leon/Documents/Dateien/Privat/Projekte/Bachelorarbeit/Project/MIRMI_2DOF_Drawing_Bot/code/api/drawing_bot_api/trajectory_optimizer/simulator.py:88: RuntimeWarning: invalid value encountered in multiply
  _points_y = points[:, 1:] + normals[:, 1:] * phase_offsets
/Users/leon/Documents/Dateien/Privat/Projekte/Bachelorarbeit/Project/MIRMI_2DOF_Drawing_Bot/code/api/venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarni

generation: 25	best_set: 1	0.7708095059696782	sigma: 0.905197094668345	best: [0.40022908 0.10612275 2.70810648 0.91860767]
generation: 26	best_set: 1	0.7046419209950847	sigma: 0.8870931527749781	best: [ 0.29141407 -0.63615825  3.18184686 -0.69053234]
generation: 27	best_set: 7	0.6722288205409341	sigma: 0.8693512897194784	best: [-0.0384068   0.23459834  2.44204096 -0.16561955]
generation: 28	best_set: 0	0.6803605197201266	sigma: 0.8519642639250888	best: [-0.0384068   0.23459834  2.44204096 -0.16561955]
generation: 29	best_set: 2	0.6896432763197857	sigma: 0.8349249786465871	best: [-0.79865874  1.16047419  2.75954309  0.61646525]
generation: 30	best_set: 2	0.6601665269746799	sigma: 0.8182264790736553	best: [-0.10445887  0.43067595  2.08689483  2.14465341]
generation: 31	best_set: 6	0.74075777568845	sigma: 0.8018619494921821	best: [0.22520205 1.48432295 0.94279523 1.41959623]
generation: 32	best_set: 8	0.7146456314464127	sigma: 0.7858247105023386	best: [-0.11870102  0.43372254  3.91068574 